In [ ]:
{
    
    "DAU (R1)": """SELECT
    '#Overall#' as "App", 
    1 as "Rolling",
    Overall as "DAU", 
    Overall-LastOverall as "WoW -7d", (Overall-LastOverall)/LastOverall as "WoW% -7d", 
    Overall-Last2_Overall as "WoW -14d", (Overall-Last2_Overall)/Last2_Overall as "WoW% -14d"
FROM (
    SELECT  
        sum(case when metrics_date = last_date then Overall else 0 end) as Overall,
        sum(case when metrics_date = to_char(cast(cast(last_date as timestamp) + '-7 d' as date), 'yyyy-mm-dd') then Overall else 0 end) as LastOverall,
        sum(case when metrics_date = to_char(cast(cast(last_date as timestamp) + '-14 d' as date), 'yyyy-mm-dd') then Overall else 0 end) as Last2_Overall
    FROM (
        SELECT 
            metrics_date,
            CASE WHEN cast(${__to:date:'YYYY-MM-DD'} as TEXT)>='${DauMetricsDate}' THEN cast('${DauMetricsDate}' as TEXT) ELSE cast(${__to:date:'YYYY-MM-DD'} as TEXT) END as last_date,
            SUM(CAST(metrics->'ActiveUser'->'uu'->>'uu' AS DOUBLE PRECISION)) as Overall
        FROM sapphire.sapphire_engagement_metrics_master
        WHERE metrics_date>= to_char(cast(cast(${__from:date:'YYYY-MM-DD'} as timestamp) + '- 14 DAY' as date), 'yyyy-mm-dd')::TEXT AND metrics_date <= ${__to:date:'YYYY-MM-DD'}
              AND application_id in ('2130688B018F4B44BBED68E7A42BBA1E','AE427635ADC245AE973038BCB3D7C21B','4DC5714ABCAD449BA13A9B701A3CF296','4A5B528B59954AAE8725B509A41FBF1A','F185A93DE6764B098D55089F610A3FB8','FC320C411FC12CD4DFBE9A00F3161364')
              AND period = 1
              AND market = '#Overall#'
              AND os_version = '#Overall#'
              AND device_model = '#Overall#'
              AND client_version = '#Overall#'
              AND client_build_type = '#Overall#'
              AND install_channel_l1 = '#Overall#'
              AND install_channel_l2 = '#Overall#'
              AND install_channel_l3 = '#Overall#'
              AND install_channel_l4 = '#Overall#'
              AND mini_app = ''
              AND first_launch_source = '#Overall#'
              AND launch_source = '#Overall#'
        GROUP BY 
            metrics_date
    ) as a
) as b
""",
"DAU (R7)": """SELECT
    '#Overall#' as "App", 
    7 as "Rolling",
    Overall as "DAU", 
    Overall-LastOverall as "WoW -7d", (Overall-LastOverall)/LastOverall as "WoW% -7d", 
    Overall-Last2_Overall as "WoW -14d", (Overall-Last2_Overall)/Last2_Overall as "WoW% -14d"
FROM (
    SELECT  
        sum(case when metrics_date = last_date then Overall else 0 end) as Overall,
        sum(case when metrics_date = to_char(cast(cast(last_date as timestamp) + '-7 d' as date), 'yyyy-mm-dd') then Overall else 0 end) as LastOverall,
        sum(case when metrics_date = to_char(cast(cast(last_date as timestamp) + '-14 d' as date), 'yyyy-mm-dd') then Overall else 0 end) as Last2_Overall
    FROM (
                select metrics_date, last_date, AVG("DAU") OVER (ORDER BY metrics_date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS Overall
                from 
                (
                SELECT 
                    metrics_date, CASE WHEN cast(${__to:date:'YYYY-MM-DD'} as TEXT)>='${DauMetricsDate}' THEN cast('${DauMetricsDate}' as TEXT) ELSE cast(${__to:date:'YYYY-MM-DD'} as TEXT) END as last_date, 
                    SUM(CAST(metrics->'ActiveUser'->'uu'->>'uu' AS DOUBLE PRECISION)) as "DAU"
                FROM sapphire.sapphire_engagement_metrics_master
                WHERE metrics_date>= to_char(cast(cast(${__from:date:'YYYY-MM-DD'} as timestamp) + '- 14 DAY' as date), 'yyyy-mm-dd')::TEXT AND metrics_date <= ${__to:date:'YYYY-MM-DD'}
                  AND application_id in ('2130688B018F4B44BBED68E7A42BBA1E','AE427635ADC245AE973038BCB3D7C21B','4DC5714ABCAD449BA13A9B701A3CF296','4A5B528B59954AAE8725B509A41FBF1A','F185A93DE6764B098D55089F610A3FB8','FC320C411FC12CD4DFBE9A00F3161364')
                  AND period = 1
                  AND market = '#Overall#'
                  AND os_version = '#Overall#'
                  AND device_model = '#Overall#'
                  AND client_version = '#Overall#'
                  AND client_build_type = '#Overall#'
                  AND install_channel_l1 = '#Overall#'
                  AND install_channel_l2 = '#Overall#'
                  AND install_channel_l3 = '#Overall#'
                  AND install_channel_l4 = '#Overall#'
                  AND mini_app = ''
                  AND first_launch_source = '#Overall#'
                  AND launch_source = '#Overall#'
                Group by metrics_date

                ) t 

        ) as a
    ) as b""",
    "User Retention 7d - all markets": """SELECT
     '#Overall#' as "App",  
     1 as "Rolling",
     Overall as "Retention D7",  
    (Overall-LastOverall)*100 as "WoW -7d", (Overall-LastOverall)/LastOverall as "WoW% -7d",
    (Overall-Last2_Overall)*100 as "WoW -14d", (Overall-Last2_Overall)/Last2_Overall as "WoW% -14d"
FROM (
    SELECT
        sum(case when metrics_date = last_date then Overall else 0 end) as Overall,
        sum(case when metrics_date = to_char(cast(cast(last_date as timestamp) + '-7 d' as date), 'yyyy-mm-dd') then Overall else 0 end) as LastOverall,
        sum(case when metrics_date = to_char(cast(cast(last_date as timestamp) + '-14 d' as date), 'yyyy-mm-dd') then Overall else 0 end) as Last2_Overall
    FROM (
            SELECT metrics_date, 
            CASE WHEN cast(${__to:date:'YYYY-MM-DD'} as TEXT)>=to_char(cast(cast('${$RetentionMetricsDate}' as timestamp) + '-7 d' as date), 'yyyy-mm-dd')  
                THEN to_char(cast(cast('${$RetentionMetricsDate}' as timestamp) + '-7 d' as date), 'yyyy-mm-dd') 
                ELSE cast(${__to:date:'YYYY-MM-DD'} as TEXT) END as last_date,
                sum(cast(device_retention_rate_new_daily->>'return_count' as DOUBLE PRECISION)) / sum(cast(device_retention_rate_new_daily->>'dau' as DOUBLE PRECISION)) as Overall
            FROM sapphire.sapphire_retention_master
            WHERE metrics_date>= to_char(cast(cast(${__from:date:'YYYY-MM-DD'} as timestamp) + '- 21 DAY' as date), 'yyyy-mm-dd')::TEXT AND metrics_date <= ${__to:date:'YYYY-MM-DD'}
                AND offset_day = 7
                AND application_id in ('2130688B018F4B44BBED68E7A42BBA1E','AE427635ADC245AE973038BCB3D7C21B','4DC5714ABCAD449BA13A9B701A3CF296','4A5B528B59954AAE8725B509A41FBF1A','F185A93DE6764B098D55089F610A3FB8','FC320C411FC12CD4DFBE9A00F3161364')
                AND market = '#Overall#'
                -- AND market in ('en-US','en-GB','de-DE','fr-FR','en-AU','en-CA','fr-CA')
                AND clientversion = '#Overall#'
                AND install_source in ('Organic', 'FirstParty', 'RewardsUpsell', 'PaidAds', 'Google Ads ACI', 'Upsell')
                AND osversion = '#Overall#'
                AND build = '#Overall#'
                AND mini_app = ''
                AND launch_source = '#Overall#'
                AND device_model = '#Overall#'
            GROUP BY metrics_date
    ) as a
) as b""",
    "User Retention 7d rolling avg - all markets": """SELECT
     '#Overall#' as "App",  
     7 as "Rolling",
     Overall as "Retention D7",  
    (Overall-LastOverall)*100 as "WoW -7d", (Overall-LastOverall)/LastOverall as "WoW% -7d",
    (Overall-Last2_Overall)*100 as "WoW -14d", (Overall-Last2_Overall)/Last2_Overall as "WoW% -14d"
FROM (
    SELECT
        sum(case when metrics_date = last_date then Overall else 0 end) as Overall,
        sum(case when metrics_date = to_char(cast(cast(last_date as timestamp) + '-7 d' as date), 'yyyy-mm-dd') then Overall else 0 end) as LastOverall,
        sum(case when metrics_date = to_char(cast(cast(last_date as timestamp) + '-14 d' as date), 'yyyy-mm-dd') then Overall else 0 end) as Last2_Overall
    FROM (

            select metrics_date, last_date, AVG("retention") OVER (ORDER BY metrics_date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS Overall
            from 
            (
                SELECT metrics_date,
                            CASE WHEN cast(${__to:date:'YYYY-MM-DD'} as TEXT)>=to_char(cast(cast('${$RetentionMetricsDate}' as timestamp) + '-7 d' as date), 'yyyy-mm-dd')  
                THEN to_char(cast(cast('${$RetentionMetricsDate}' as timestamp) + '-7 d' as date), 'yyyy-mm-dd') 
                ELSE cast(${__to:date:'YYYY-MM-DD'} as TEXT) END as last_date,
                    sum(cast(device_retention_rate_new_daily->>'return_count' as DOUBLE PRECISION)) / sum(cast(device_retention_rate_new_daily->>'dau' as DOUBLE PRECISION)) as "retention"
                FROM sapphire.sapphire_retention_master
                WHERE metrics_date>= to_char(cast(cast(${__from:date:'YYYY-MM-DD'} as timestamp) + '- 28 DAY' as date), 'yyyy-mm-dd')::TEXT 
                AND metrics_date <= to_char(cast(cast('${RetentionMetricsDate}' as timestamp) + '-7 d' as date), 'yyyy-mm-dd')::TEXT 
                    AND offset_day = 7
                    AND application_id in ('2130688B018F4B44BBED68E7A42BBA1E','AE427635ADC245AE973038BCB3D7C21B','4DC5714ABCAD449BA13A9B701A3CF296','4A5B528B59954AAE8725B509A41FBF1A', 'F185A93DE6764B098D55089F610A3FB8', 'FC320C411FC12CD4DFBE9A00F3161364')
                    AND market = '#Overall#'
                    -- AND market in ('en-US','en-GB','de-DE','fr-FR','en-AU','en-CA','fr-CA')
                    AND clientversion = '#Overall#'
                    AND install_source in ('Organic', 'FirstParty', 'RewardsUpsell', 'PaidAds', 'Google Ads ACI', 'Upsell')
                    AND osversion = '#Overall#'
                    AND build = '#Overall#'
                    AND mini_app = ''
                    AND launch_source = '#Overall#'
                    AND device_model = '#Overall#'
                GROUP BY metrics_date
            ) t 

    ) as a
) as b
""",
    "Chat Active DAU (R1)": """SELECT
     '#Overall#' as "App",  
     1 as "Rolling",
     Overall as "Retention D7",  
    (Overall-LastOverall)*100 as "WoW -7d", (Overall-LastOverall)/LastOverall as "WoW% -7d",
    (Overall-Last2_Overall)*100 as "WoW -14d", (Overall-Last2_Overall)/Last2_Overall as "WoW% -14d"
FROM (
    SELECT
        sum(case when metrics_date = last_date then Overall else 0 end) as Overall,
        sum(case when metrics_date = to_char(cast(cast(last_date as timestamp) + '-7 d' as date), 'yyyy-mm-dd') then Overall else 0 end) as LastOverall,
        sum(case when metrics_date = to_char(cast(cast(last_date as timestamp) + '-14 d' as date), 'yyyy-mm-dd') then Overall else 0 end) as Last2_Overall
    FROM (
            SELECT metrics_date, 
            CASE WHEN cast(${__to:date:'YYYY-MM-DD'} as TEXT)>=to_char(cast(cast('${$RetentionMetricsDate}' as timestamp) + '-7 d' as date), 'yyyy-mm-dd')  
                THEN to_char(cast(cast('${$RetentionMetricsDate}' as timestamp) + '-7 d' as date), 'yyyy-mm-dd') 
                ELSE cast(${__to:date:'YYYY-MM-DD'} as TEXT) END as last_date,
                sum(cast(device_retention_rate_new_daily->>'return_count' as DOUBLE PRECISION)) / sum(cast(device_retention_rate_new_daily->>'dau' as DOUBLE PRECISION)) as Overall
            FROM sapphire.sapphire_retention_master
            WHERE metrics_date>= to_char(cast(cast(${__from:date:'YYYY-MM-DD'} as timestamp) + '- 21 DAY' as date), 'yyyy-mm-dd')::TEXT AND metrics_date <= ${__to:date:'YYYY-MM-DD'}
                AND offset_day = 7
                AND application_id in ('2130688B018F4B44BBED68E7A42BBA1E','AE427635ADC245AE973038BCB3D7C21B','4DC5714ABCAD449BA13A9B701A3CF296','4A5B528B59954AAE8725B509A41FBF1A','F185A93DE6764B098D55089F610A3FB8','FC320C411FC12CD4DFBE9A00F3161364')
                AND market = '#Overall#'
                -- AND market in ('en-US','en-GB','de-DE','fr-FR','en-AU','en-CA','fr-CA')
                AND clientversion = '#Overall#'
                AND install_source in ('Organic', 'FirstParty', 'RewardsUpsell', 'PaidAds', 'Google Ads ACI', 'Upsell')
                AND osversion = '#Overall#'
                AND build = '#Overall#'
                AND mini_app = ''
                AND launch_source = '#Overall#'
                AND device_model = '#Overall#'
            GROUP BY metrics_date
    ) as a
) as b""",
    "Chat Active DAU (R7)": """SELECT
    '#Overall#' as "App", 
    7 as "Rolling",
    Overall as "DAU", 
    Overall-LastOverall as "WoW -7d", (Overall-LastOverall)/LastOverall as "WoW% -7d", 
    Overall-Last2_Overall as "WoW -14d", (Overall-Last2_Overall)/Last2_Overall as "WoW% -14d"
FROM (
    SELECT  
        sum(case when date = last_date then Overall else 0 end) as Overall,
        sum(case when date = last_date + '-7 d' then Overall else 0 end) as LastOverall,
        sum(case when date = last_date + '-14 d' then Overall else 0 end) as Last2_Overall
    FROM
    (
    select 
    date, 
    (max(date) OVER (ORDER BY date desc)) as last_date,
    AVG("DAU") OVER (ORDER BY date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS Overall
    FROM (
SELECT 
	cast(metrics_date as timestamp) as date,
        SUM(CAST(metrics->'ActiveUser'->'dau'->'chat_active_dau' AS DOUBLE PRECISION)) AS "DAU"
FROM codex_metrics.codex_engagement_metrics_master
WHERE metrics_date >= to_char(cast(cast(${__from:date:'YYYY-MM-DD'} as timestamp) + '- 7 DAY' as date), 'yyyy-mm-dd')::TEXT 
  AND metrics_date <= ${__to:date:'YYYY-MM-DD'}
  AND application_id in ('2130688B018F4B44BBED68E7A42BBA1E','AE427635ADC245AE973038BCB3D7C21B','4DC5714ABCAD449BA13A9B701A3CF296','4A5B528B59954AAE8725B509A41FBF1A','F185A93DE6764B098D55089F610A3FB8','FC320C411FC12CD4DFBE9A00F3161364')
	AND id_type= 'reportable_id'
  AND market ='#Overall#'
  AND install_channel_l1 ='#Overall#'
  AND install_channel_l2 ='#Overall#'
	AND first_launch_entry_point ='#Overall#'
	AND period = 1
	AND os_version ='#Overall#'
	AND device_model ='#Overall#'
	AND client_version ='#Overall#'
	AND client_build_type ='#Overall#'
	AND install_channel_l3 ='#Overall#'
	AND install_channel_l4 ='#Overall#'
	AND mini_app ='#Overall#'
	AND first_launch_source ='#Overall#'
GROUP BY metrics_date
        )  a
    ) as b
) as c
CROSS JOIN generate_series(1,5)"""
}
## 这个是对的数